In [11]:
# **IMPORTANDO LIBRERIAS**

In [12]:
import sys
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy import create_engine, Column, Integer, String, VARCHAR,Boolean, TIMESTAMP
from sqlalchemy.types import DateTime
from sqlalchemy.orm import sessionmaker


sys.path.append(os.path.abspath(os.path.join('..')))

In [13]:
load_dotenv()  # Carga las variables del archivo .env

config = {
    'host': os.getenv('HOST'),
    'user': os.getenv('USER'),
    'password': os.getenv('PASSWORD'),
    'database': os.getenv('DATABASE'),
    'port': os.getenv('PORT')
}

In [14]:
# Crear la URL de conexión para SQLAlchemy
db_url = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/caso_7"

# Crear el motor de SQLAlchemy
engine = create_engine(db_url, echo=True)



In [15]:
Base = declarative_base()


# **1. GENERACIÓN DE TABLAS**

In [16]:
class ProductHierarchy(Base):

    __tablename__ = 'product_hierarchy'
    __table_args__= {'extend_existing':True}

    #| ------------------ Columnas ----------------|

    id =Column(Integer, primary_key=True)
    parent_id = Column(Integer)
    level_text = Column(VARCHAR(19))
    level_name = Column(VARCHAR(8))


class ProductPrices(Base):

    __tablename__ = 'product_prices'
    __table_args__= {'extend_existing':True}

    #| ------------------ Columnas ----------------|
    
    id =Column(Integer, primary_key=True)
    product_id = Column(VARCHAR(6))
    price = Integer

class ProductDetails(Base):

    __tablename__ = 'product_details'
    __table_args__= {'extend_existing':True}
    
    #| ------------------ Columnas ----------------|

    product_id = Column(VARCHAR(6), primary_key=True)
    price = Column(Integer)
    product_name = Column(VARCHAR(32))
    category_id = Column(Integer)
    segment_id = Column(Integer)
    style_id = Column(Integer)
    category_name = Column(VARCHAR(6))
    segment_name =Column(VARCHAR(6))
    style_name = Column(VARCHAR(19))


class Sales(Base):

    __tablename__ = 'sales'
    __table_args__= {'extend_existing':True}

 #| ------------------ Columnas ----------------|
    id = Column(Integer,  primary_key=True, autoincrement=True)
    prod_id = Column(VARCHAR(6))
    qty = Column(Integer)
    price = Column(Integer)
    discount = Column(Integer)
    member = Column(Boolean)
    txn_id =Column(VARCHAR(32))
    start_txn_time = Column(TIMESTAMP(timezone=True))

Base.metadata.create_all(engine)


2025-04-29 13:42:18,034 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-04-29 13:42:18,035 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 13:42:18,036 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-04-29 13:42:18,037 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 13:42:18,038 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-04-29 13:42:18,039 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 13:42:18,040 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 13:42:18,041 INFO sqlalchemy.engine.Engine DESCRIBE `caso_7`.`product_hierarchy`
2025-04-29 13:42:18,042 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 13:42:18,044 INFO sqlalchemy.engine.Engine DESCRIBE `caso_7`.`product_prices`
2025-04-29 13:42:18,044 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 13:42:18,045 INFO sqlalchemy.engine.Engine DESCRIBE `caso_7`.`product_details`
2025-04-29 13:42:18,046 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 13:42:18,047 IN

## **1.1 CARGANDO LOS DATOS**

In [17]:
from utils import funciones_utiles

product_hierarchy_data = funciones_utiles.load_text_as_tuples('datos_tablas/product_hierarchy_data.txt')
product_prices_data = funciones_utiles.load_text_as_tuples('datos_tablas/product_prices_data.txt')
product_details_data =funciones_utiles.load_text_as_tuples('datos_tablas/product_details_data.txt')
sales_data = funciones_utiles.load_text_as_tuples('datos_tablas/sales_data.txt')


In [18]:
# Creando la session
Session = sessionmaker(bind=engine)
session = Session()

In [19]:
product_hierarchy_data

[('1', None, 'Womens', 'Category'),
 ('2', None, 'Mens', 'Category'),
 ('3', '1', 'Jeans', 'Segment'),
 ('4', '1', 'Jacket', 'Segment'),
 ('5', '2', 'Shirt', 'Segment'),
 ('6', '2', 'Socks', 'Segment'),
 ('7', '3', 'Navy Oversized', 'Style'),
 ('8', '3', 'Black Straight', 'Style'),
 ('9', '3', 'Cream Relaxed', 'Style'),
 ('10', '4', 'Khaki Suit', 'Style'),
 ('11', '4', 'Indigo Rain', 'Style'),
 ('12', '4', 'Grey Fashion', 'Style'),
 ('13', '5', 'White Tee', 'Style'),
 ('14', '5', 'Teal Button Up', 'Style'),
 ('15', '5', 'Blue Polo', 'Style'),
 ('16', '6', 'Navy Solid', 'Style'),
 ('17', '6', 'White Striped', 'Style'),
 ('18', '6', 'Pink Fluro Polkadot', 'Style')]

### **1.1.1 Cargando datos `product_hierarchy`**

In [20]:

objects = [
    ProductHierarchy(

        id =int(id),
        parent_id = int(parent_id) if parent_id is not None else None,
        level_text = level_text,
        level_name = level_name
    )
    for id, parent_id, level_text, level_name in product_hierarchy_data
]

session.bulk_save_objects(objects)
session.commit()

2025-04-29 13:42:18,478 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 13:42:18,479 INFO sqlalchemy.engine.Engine INSERT INTO product_hierarchy (id, level_text, level_name) VALUES (%(id)s, %(level_text)s, %(level_name)s)
2025-04-29 13:42:18,480 INFO sqlalchemy.engine.Engine [generated in 0.00093s] [{'id': 1, 'level_text': 'Womens', 'level_name': 'Category'}, {'id': 2, 'level_text': 'Mens', 'level_name': 'Category'}]
2025-04-29 13:42:18,488 INFO sqlalchemy.engine.Engine INSERT INTO product_hierarchy (id, parent_id, level_text, level_name) VALUES (%(id)s, %(parent_id)s, %(level_text)s, %(level_name)s)
2025-04-29 13:42:18,489 INFO sqlalchemy.engine.Engine [generated in 0.00060s] [{'id': 3, 'parent_id': 1, 'level_text': 'Jeans', 'level_name': 'Segment'}, {'id': 4, 'parent_id': 1, 'level_text': 'Jacket', 'level_name': 'Segment'}, {'id': 5, 'parent_id': 2, 'level_text': 'Shirt', 'level_name': 'Segment'}, {'id': 6, 'parent_id': 2, 'level_text': 'Socks', 'level_name': 'Segment'}, {'

### **1.1.2 Cargando datos `ProductPrices`**

In [21]:

objects = [
    ProductPrices(

        id =int(id),
        product_id = product_id,
        price = int(price)
    )
    for id, product_id, price in product_prices_data
]

session.bulk_save_objects(objects)
session.commit()

2025-04-29 13:42:18,508 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 13:42:18,509 INFO sqlalchemy.engine.Engine INSERT INTO product_prices (id, product_id) VALUES (%(id)s, %(product_id)s)
2025-04-29 13:42:18,510 INFO sqlalchemy.engine.Engine [generated in 0.00093s] [{'id': 7, 'product_id': 'c4a632'}, {'id': 8, 'product_id': 'e83aa3'}, {'id': 9, 'product_id': 'e31d39'}, {'id': 10, 'product_id': 'd5e9a6'}, {'id': 11, 'product_id': '72f5d4'}, {'id': 12, 'product_id': '9ec847'}, {'id': 13, 'product_id': '5d267b'}, {'id': 14, 'product_id': 'c8d436'}  ... displaying 10 of 12 total bound parameter sets ...  {'id': 17, 'product_id': 'b9a74d'}, {'id': 18, 'product_id': '2feb6b'}]
2025-04-29 13:42:18,513 INFO sqlalchemy.engine.Engine COMMIT


### **1.1.3 Cargando datos `ProductDetails`**

In [22]:
objects = [
    ProductDetails(

        product_id = product_id, 
        price = int(price),
        product_name = product_name,
        category_id = int(category_id),
        segment_id = int(segment_id),
        style_id = int(style_id),
        category_name = category_name,
        segment_name =segment_name,
        style_name = style_name,
    )
    for product_id, price, product_name, category_id, segment_id,  style_id, category_name,  segment_name, style_name in product_details_data
]

session.bulk_save_objects(objects)
session.commit()

2025-04-29 13:42:18,540 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 13:42:18,541 INFO sqlalchemy.engine.Engine INSERT INTO product_details (product_id, price, product_name, category_id, segment_id, style_id, category_name, segment_name, style_name) VALUES (%(product_id)s, %(price)s, %(product_name)s, %(category_id)s, %(segment_id)s, %(style_id)s, %(category_name)s, %(segment_name)s, %(style_name)s)
2025-04-29 13:42:18,542 INFO sqlalchemy.engine.Engine [generated in 0.00081s] [{'product_id': 'c4a632', 'price': 13, 'product_name': 'Navy Oversized Jeans - Womens', 'category_id': 1, 'segment_id': 3, 'style_id': 7, 'category_name': 'Womens', 'segment_name': 'Jeans', 'style_name': 'Navy Oversized'}, {'product_id': 'e83aa3', 'price': 32, 'product_name': 'Black Straight Jeans - Womens', 'category_id': 1, 'segment_id': 3, 'style_id': 8, 'category_name': 'Womens', 'segment_name': 'Jeans', 'style_name': 'Black Straight'}, {'product_id': 'e31d39', 'price': 10, 'product_name': 'Cream 

### **1.1.3 Cargando datos `Sales`**

In [23]:
objects = [
    Sales(

        prod_id = prod_id,
        qty = int(qty),
        price = int(price),
        discount = int(discount),
        member =True if member == 't' else False,
        txn_id =txn_id,
        start_txn_time = start_txn_time,
    )
    for prod_id, qty, price, discount, member, txn_id,  start_txn_time in sales_data
]

session.bulk_save_objects(objects)
session.commit()

2025-04-29 13:42:18,900 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 13:42:19,002 INFO sqlalchemy.engine.Engine INSERT INTO sales (prod_id, qty, price, discount, `member`, txn_id, start_txn_time) VALUES (%(prod_id)s, %(qty)s, %(price)s, %(discount)s, %(member)s, %(txn_id)s, %(start_txn_time)s)
2025-04-29 13:42:19,003 INFO sqlalchemy.engine.Engine [generated in 0.04885s] [{'prod_id': 'c4a632', 'qty': 4, 'price': 13, 'discount': 17, 'member': 1, 'txn_id': '54f307', 'start_txn_time': '2021-02-13 01:59:43.296'}, {'prod_id': '5d267b', 'qty': 4, 'price': 40, 'discount': 17, 'member': 1, 'txn_id': '54f307', 'start_txn_time': '2021-02-13 01:59:43.296'}, {'prod_id': 'b9a74d', 'qty': 4, 'price': 17, 'discount': 17, 'member': 1, 'txn_id': '54f307', 'start_txn_time': '2021-02-13 01:59:43.296'}, {'prod_id': '2feb6b', 'qty': 2, 'price': 29, 'discount': 17, 'member': 1, 'txn_id': '54f307', 'start_txn_time': '2021-02-13 01:59:43.296'}, {'prod_id': 'c4a632', 'qty': 5, 'price': 13, 'discoun

<center><b><h1>PREGUNTAS</h1></b></center>

## **Análisis de Ventas de Alto Nivel**


# **1. ¿Cuál fue la cantidad total vendida de todos los productos?**

In [24]:
query = '''

    SELECT
        d.product_name,
        SUM(s.qty) AS total_vendida
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
    ORDER BY
        total_vendida DESC

'''


pd.read_sql_query(query, con=engine)

2025-04-29 13:42:19,410 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 13:42:19,411 INFO sqlalchemy.engine.Engine 

    SELECT
        d.product_name,
        SUM(s.qty) AS total_vendida
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
    ORDER BY
        total_vendida DESC


2025-04-29 13:42:19,411 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 13:42:19,458 INFO sqlalchemy.engine.Engine ROLLBACK


,product_name,total_vendida
0,Grey Fashion Jacket - Womens,3876.0
1,Navy Oversized Jeans - Womens,3856.0
2,Blue Polo Shirt - Mens,3819.0
3,White Tee Shirt - Mens,3800.0
4,Navy Solid Socks - Mens,3792.0
5,Black Straight Jeans - Womens,3786.0
6,Pink Fluro Polkadot Socks - Mens,3770.0
7,Indigo Rain Jacket - Womens,3757.0
8,Khaki Suit Jacket - Womens,3752.0
9,Cream Relaxed Jeans - Womens,3707.0


# **2. ¿Cuál es el ingreso total generado por todos los productos antes de los descuentos?**


In [28]:
query = '''

    SELECT
        d.product_name,
        SUM(s.qty) * SUM(s.price) as total_ingresos
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
   

'''


pd.read_sql_query(query, con=engine)

2025-04-29 13:44:08,364 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 13:44:08,365 INFO sqlalchemy.engine.Engine 

    SELECT
        d.product_name,
        SUM(s.qty) * SUM(s.price) as total_ingresos
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
   


2025-04-29 13:44:08,365 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 13:44:08,404 INFO sqlalchemy.engine.Engine ROLLBACK


,product_name,total_ingresos
0,Navy Oversized Jeans - Womens,63863072.0
1,White Tee Shirt - Mens,192736000.0
2,White Striped Socks - Mens,77233805.0
3,Pink Fluro Polkadot Socks - Mens,137537140.0
4,Cream Relaxed Jeans - Womens,46078010.0
5,Indigo Rain Jacket - Womens,89228750.0
6,Blue Polo Shirt - Mens,276022044.0
7,Navy Solid Socks - Mens,174871872.0
8,Black Straight Jeans - Womens,150955392.0
9,Khaki Suit Jacket - Womens,107611112.0


# **3.¿Cuál fue el monto total de descuento para todos los productos?**


In [26]:
query = '''

    SELECT
        d.product_name,
        SUM(s.qty * s.price * s.discount/100) as total_descuentos
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
   

'''


pd.read_sql_query(query, con=engine)

2025-04-29 13:42:19,551 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 13:42:19,552 INFO sqlalchemy.engine.Engine 

    SELECT
        d.product_name,
        SUM(s.qty * s.price * s.discount/100) as total_descuentos
    FROM 
        sales s
    INNER JOIN
        product_details d
        ON
        s.prod_id = d.product_id
    GROUP BY
        d.product_name
   


2025-04-29 13:42:19,553 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 13:42:19,597 INFO sqlalchemy.engine.Engine ROLLBACK


,product_name,total_descuentos
0,Navy Oversized Jeans - Womens,6135.61
1,White Tee Shirt - Mens,18377.60
2,White Striped Socks - Mens,7410.81
3,Pink Fluro Polkadot Socks - Mens,12952.27
4,Cream Relaxed Jeans - Womens,4463.40
5,Indigo Rain Jacket - Womens,8642.53
6,Blue Polo Shirt - Mens,26819.07
7,Navy Solid Socks - Mens,16650.36
8,Black Straight Jeans - Womens,14744.96
9,Khaki Suit Jacket - Womens,10243.05


# Análisis de transacciones 

# **1. ¿Cuántas transacciones únicas hubo?**


In [30]:
query = '''

    SELECT
       COUNT(DISTINCT txn_id) AS transacciones_unicas
    FROM 
        sales

   

'''


pd.read_sql_query(query, con=engine)




2025-04-29 13:44:48,683 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 13:44:48,684 INFO sqlalchemy.engine.Engine 

    SELECT
       COUNT(DISTINCT txn_id) AS transacciones_unicas
    FROM 
        sales

   


2025-04-29 13:44:48,685 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 13:44:48,700 INFO sqlalchemy.engine.Engine ROLLBACK


,transacciones_unicas
0,2500


# **2.-¿Cuál es el promedio de productos únicos comprados en cada transacción?**

In [43]:
query = '''

    WITH cuenta_prod_unicos AS (
    SELECT
       COUNT(DISTINCT prod_id) AS cuenta_productos_unicos
    FROM 
        sales
    GROUP BY 
        txn_id )

    SELECT
        AVG(cuenta_productos_unicos)

    FROM
        cuenta_prod_unicos

'''


pd.read_sql_query(query, con=engine)




2025-04-29 14:05:31,905 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 14:05:31,906 INFO sqlalchemy.engine.Engine 

    WITH cuenta_prod_unicos AS (
    SELECT
       COUNT(DISTINCT prod_id) AS cuenta_productos_unicos
    FROM 
        sales
    GROUP BY 
        txn_id )

    SELECT
        AVG(cuenta_productos_unicos)

    FROM
        cuenta_prod_unicos


2025-04-29 14:05:31,906 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 14:05:31,925 INFO sqlalchemy.engine.Engine ROLLBACK


,AVG(cuenta_productos_unicos)
0,6.038


# **3.- ¿Cuáles son los valores del percentil 25, 50 y 75 para los ingresos por transacción?**


In [75]:
query = '''

WITH total_ventas AS (
    SELECT
        txn_id,
        SUM(qty * price) AS total_sales
    FROM
        sales
    GROUP BY
        txn_id
),
ventas_rankeadas AS (
    SELECT
        total_sales,
        ROW_NUMBER() OVER (ORDER BY total_sales) AS row_num,
        COUNT(*) OVER () AS total_count
    FROM total_ventas
)
SELECT
    ROUND(AVG(CASE WHEN row_num = CEIL(0.25 * total_count) THEN total_sales END), 2) AS percentile_25,
    ROUND(AVG(CASE WHEN row_num = CEIL(0.50 * total_count) THEN total_sales END), 2) AS percentile_50,
    ROUND(AVG(CASE WHEN row_num = CEIL(0.75 * total_count) THEN total_sales END), 2) AS percentile_75
FROM ventas_rankeadas;

'''


pd.read_sql_query(query, con=engine)




2025-04-29 14:32:17,724 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 14:32:17,724 INFO sqlalchemy.engine.Engine 

WITH total_ventas AS (
    SELECT
        txn_id,
        SUM(qty * price) AS total_sales
    FROM
        sales
    GROUP BY
        txn_id
),
ventas_rankeadas AS (
    SELECT
        total_sales,
        ROW_NUMBER() OVER (ORDER BY total_sales) AS row_num,
        COUNT(*) OVER () AS total_count
    FROM total_ventas
)
SELECT
    ROUND(AVG(CASE WHEN row_num = CEIL(0.25 * total_count) THEN total_sales END), 2) AS percentile_25,
    ROUND(AVG(CASE WHEN row_num = CEIL(0.50 * total_count) THEN total_sales END), 2) AS percentile_50,
    ROUND(AVG(CASE WHEN row_num = CEIL(0.75 * total_count) THEN total_sales END), 2) AS percentile_75
FROM ventas_rankeadas;


2025-04-29 14:32:17,725 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 14:32:17,750 INFO sqlalchemy.engine.Engine ROLLBACK


,percentile_25,percentile_50,percentile_75
0,375.0,509.0,647.0


 # **3. ¿Cuál es el valor promedio del descuento por transacción?**


In [80]:
query = '''
WITH total_descuento_por_transaccion AS (
SELECT
    txn_id, 
    SUM(qty * price * discount/100) as total_descuentos

FROM
    sales
GROUP BY
    txn_id )

SELECT
    ROUND(AVG(total_descuentos)) AS promedio_dscto_transaccion
FROM 
    total_descuento_por_transaccion

'''


pd.read_sql_query(query, con=engine)




2025-04-29 14:33:35,057 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-29 14:33:35,058 INFO sqlalchemy.engine.Engine 
WITH total_descuento_por_transaccion AS (
SELECT
    txn_id, 
    SUM(qty * price * discount/100) as total_descuentos

FROM
    sales
GROUP BY
    txn_id )

SELECT
    ROUND(AVG(total_descuentos)) AS promedio_dscto_transaccion
FROM 
    total_descuento_por_transaccion


2025-04-29 14:33:35,058 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-29 14:33:35,084 INFO sqlalchemy.engine.Engine ROLLBACK


,promedio_dscto_transaccion
0,62.0
